In [18]:
// Load the EnvConfigHelper class from our project
#load "../EnvConfigHelper.cs"

  // Load configuration from .env file
  var modelDeploymentName = EnvConfigHelper.AzureOpenAIDeploymentName;
  var azureOpenAIEndpoint = EnvConfigHelper.AzureOpenAIEndpoint;
  var azureOpenAIKey = EnvConfigHelper.AzureOpenAIKey;


// Display loaded configuration (without sensitive data)
Console.WriteLine("🔑 Configuration loaded successfully!");
Console.WriteLine($"Azure OpenAI Endpoint: {azureOpenAIEndpoint}");
Console.WriteLine($"Deployment Name: {modelDeploymentName}");
Console.WriteLine($"API Key: {(string.IsNullOrEmpty(azureOpenAIKey) ? "❌ Not loaded" : "✅ Loaded (hidden)")}");

✅ Loaded .env from: d:\VSC\semantic-kernel-in-action-1-fundamentals-3836112\src\.env
🔑 Configuration loaded successfully!
Azure OpenAI Endpoint: https://dis-openai-0705.openai.azure.com/
Deployment Name: gpt-4.1
API Key: ✅ Loaded (hidden)


In [ ]:
#r "nuget: Microsoft.SemanticKernel, *"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, *"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using System.ComponentModel;
using System.IO;
using System.Text.Json;
using System.Net.Http;



Installed Packages Microsoft.SemanticKernel, 1.61.0 Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.61.0

In [22]:
async Task Execute()
{
    try
    {
        // Validate Azure OpenAI configuration
        if (string.IsNullOrEmpty(azureOpenAIKey) || string.IsNullOrEmpty(azureOpenAIEndpoint))
        {
            Console.WriteLine("❌ Azure OpenAI configuration missing. Check your .env file.");
            return;
        }

        var kernelBuilder = Kernel.CreateBuilder();
        
        // Use correct model ID (gpt-4 instead of gpt-4.1)
        kernelBuilder.Services.AddAzureOpenAIChatCompletion(
            modelDeploymentName,
            azureOpenAIEndpoint,
            azureOpenAIKey,
            modelId: "gpt-4" // Fixed model ID
        );

        // Verify plugin directory exists
        var pluginDirectory = Path.Combine(
            Directory.GetCurrentDirectory(),
            "plugins",
            "RoleTalk"
        );
        
        if (!Directory.Exists(pluginDirectory))
        {
            Console.WriteLine($"❌ Plugin directory not found: {pluginDirectory}");
            return;
        }

        var kernel = kernelBuilder.Build();
        kernel.ImportPluginFromPromptDirectory(pluginDirectory);

        Console.WriteLine($"📁 Loaded plugins from: {pluginDirectory}");
        Console.WriteLine($"🔌 Available plugins: {string.Join(", ", kernel.Plugins.Select(p => p.Name))}");

        string question = "What's the best way to deal with a city-wide power outage?";
        Console.WriteLine($"❓ Question: {question}\n");

        var chainingTemplate = @"
{{set ""responseAsPoliceman"" (RoleTalk-RespondAsPoliceman input) }}
{{set ""responseAsScientist"" (RoleTalk-RespondAsScientist input) }}
{{set ""opinionFromScientificToPoliceman"" (RoleTalk-RespondAsScientist responseAsPoliceman) }}

{{!-- Create structured output --}}
{
  ""question"": ""{{input}}"",
  ""responses"": {
    ""policeman"": ""{{json responseAsPoliceman}}"",
    ""scientist"": ""{{json responseAsScientist}}"",
    ""scientistOpinionOnPoliceman"": ""{{json opinionFromScientificToPoliceman}}""
  }
}";

        var chainingFunction = kernel.CreateFunctionFromPrompt(
            new PromptTemplateConfig()
            {
                Template = chainingTemplate,
                TemplateFormat = "handlebars",
                Name = "MultiPersonaResponse"
            },
            new HandlebarsPromptTemplateFactory()
        );

        Console.WriteLine("🤖 Processing multi-persona responses...");
        
        var result = await kernel.InvokeAsync(
            chainingFunction,
            new KernelArguments { ["input"] = question }
        );

        Console.WriteLine("📋 Results:");
        Console.WriteLine(new string('=', 60));
        
        // Try to parse and pretty-print JSON if possible
        try
        {
            var jsonResult = result.GetValue<string>();
            var parsedJson = JsonDocument.Parse(jsonResult);
            var prettyJson = JsonSerializer.Serialize(parsedJson, new JsonSerializerOptions 
            { 
                WriteIndented = true 
            });
            Console.WriteLine(prettyJson);
        }
        catch (JsonException)
        {
            // If JSON parsing fails, just output the raw result
            Console.WriteLine(result.GetValue<string>());
        }
    }
    catch (DirectoryNotFoundException ex)
    {
        Console.WriteLine($"📁 Directory Error: {ex.Message}");
    }
    catch (HttpRequestException ex)
    {
        Console.WriteLine($"🌐 Network Error: {ex.Message}");
        Console.WriteLine("Check your Azure OpenAI endpoint and network connection.");
    }
    catch (UnauthorizedAccessException ex)
    {
        Console.WriteLine($"🔐 Authentication Error: {ex.Message}");
        Console.WriteLine("Verify your Azure OpenAI API key.");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error: {ex.Message}");
        if (ex.InnerException != null)
        {
            Console.WriteLine($"   Inner Exception: {ex.InnerException.Message}");
        }
    }
}

In [15]:
async Task Execute()
  {

    var kernelBuilder = Kernel.CreateBuilder();
    kernelBuilder.Services.AddAzureOpenAIChatCompletion(
        modelDeploymentName,
        azureOpenAIEndpoint,
        azureOpenAIKey,
        modelId: "gpt-4.1"
    );
    var pluginDirectory = Path.Combine(
        System.IO.Directory.GetCurrentDirectory(),
        "plugins",
        "RoleTalk");
    var kernel = kernelBuilder.Build();
    kernel.ImportPluginFromPromptDirectory(pluginDirectory);

    string question = "What's the best way to deal with a city-wide power outage?";
    var chainingFunctionsWithHandlebarsFunction = kernel.CreateFunctionFromPrompt(
        new()
        {
          Template = @"
                {{set ""responseAsPoliceman"" (RoleTalk-RespondAsPoliceman input) }}
                {{set ""responseAsScientific"" (RoleTalk-RespondAsScientist input) }}
                {{set ""opinionFromScientificToPoliceman"" (RoleTalk-RespondAsScientist responseAsPoliceman) }}

                {{!-- Example of concatenating text and variables to finally output it with json --}}
                {{set ""finalOutput"" (concat ""Policeman: "" responseAsPoliceman "" Scientific: "" responseAsScientific  "" Scientific to Policeman: "" opinionFromScientificToPoliceman)}}
                
                Output the following responses as is, do not modify anything:
                {{json finalOutput}}
                ",
          TemplateFormat = "handlebars"
        },
        new HandlebarsPromptTemplateFactory()
    );

    var resp =
        await kernel.InvokeAsync(
            chainingFunctionsWithHandlebarsFunction,
            new() {
                    { "input", question }
            });

    Console.WriteLine($"Result:  {resp}");

    Console.ReadLine();
  }

In [23]:
await Execute();

📁 Loaded plugins from: d:\VSC\semantic-kernel-in-action-1-fundamentals-3836112\src\03_10e\plugins\RoleTalk
🔌 Available plugins: RoleTalk
❓ Question: What's the best way to deal with a city-wide power outage?

🤖 Processing multi-persona responses...
📋 Results:
Certainly! Here is a structured summary and analysis based on your input, showing the distinct perspectives and thoughtful reflection between a policeman and a scientist responding to a city-wide power outage:

---

**Policeman’s Perspective:**

The policeman emphasizes practical, immediate steps for public safety during a city-wide power outage:
- Stay calm and keep informed using battery-powered devices.
- Have a flashlight and basic supplies.
- Avoid unnecessary travel, as traffic signals are down and roads may be hazardous.
- Report suspicious activity or emergencies to the police, who are patrolling the area.
- Check in on vulnerable neighbors, especially the elderly or those needing extra assistance.
- Promotes community sol